In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [34, 12767, 18155, 2675, 4254, 3426, 10488, 2296, 10946, 6616, 5739, 7988, 1119, 14772, 4325, 13934, 1538, 14969, 7643, 7212, 19597, 1348, 9519, 13691, 4719, 16812, 11823, 4475, 1149, 19647, 9930, 2420, 9359, 15802, 9188, 8223, 4270, 19784, 16853, 11482, 12380, 9106, 1051, 1044, 12115, 2741, 11659, 8856, 14304, 10032, 17197, 5119, 7210, 17054, 9364, 6972, 11728, 10724, 11430, 9727, 9342, 7125, 9938, 3850, 8398, 12795, 15237, 12545, 18119, 11420, 3976, 121, 19744, 9908, 13157, 14760, 8914, 2252, 1294, 13759, 12515, 36, 16232, 11061, 152, 575, 19130, 10352, 12640, 16477, 17998, 16357, 5771, 13892, 17946, 15697, 18801, 8260, 5352, 12612, 355, 7688, 3735, 10142, 18805, 3370, 16485, 14147, 10371, 14143, 2931, 12449, 19791, 4095, 2338, 5988, 20069, 3230, 14682, 11345, 10196, 12760, 13109, 14614, 17132, 15925, 18029, 6884, 15579, 19625, 15092, 4024, 13416, 19516, 19059, 671, 11874, 9740, 9673, 9056, 17392, 18182, 17458, 11299, 12373, 65

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'embed_sm', train_mode = '')

Epoch [1/500], Loss: 1.3866, Accuracy: 0.2708, Edge Accuracy: []
Class 0: AUC-ROC- 0.3970, F1 Score- 0.0000; Class 1: AUC-ROC- 0.6337, F1 Score- 0.0000; Class 2: AUC-ROC- 0.5787, F1 Score- 0.0000; Class 3: AUC-ROC- 0.3820, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.4978; Macro-Average F1 Score: 0.0000
Validation Loss: 1.3855, Validation Accuracy: 0.3150, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.4625, F1 Score- 0.0000; Class 1: AUC-ROC- 0.7113, F1 Score- 0.0000; Class 2: AUC-ROC- 0.6459, F1 Score- 0.0000; Class 3: AUC-ROC- 0.3213, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5353; Macro-Average F1 Score: 0.0000
Test Loss: 1.3846, Test Accuracy: 0.3525, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.4750, F1 Score- 0.0000; Class 1: AUC-ROC- 0.6990, F1 Score- 0.0000; Class 2: AUC-ROC- 0.6765, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4508, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5753; Macro-Average F1 Score: 0.0000

Epoch [2/500], Loss: 1.3820, Accuracy: 0.3073, Edge Accuracy: []
Class 0

In [8]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = '')

Test Loss: 0.8525, Test Accuracy: 0.9500, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.9996, F1 Score- 0.9852; Class 1: AUC-ROC- 0.9971, F1 Score- 0.9691; Class 2: AUC-ROC- 0.9971, F1 Score- 0.9050; Class 3: AUC-ROC- 0.9850, F1 Score- 0.9645; Macro-Average AUC-ROC: 0.9947; Macro-Average F1 Score: 0.9559


(0.95, 0.9947083333333333, 0.9559345551167635)